In [15]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import execute, Aer
# from qc_grader import prepare_ex3, grade_ex3
import numpy as np
from heapq import nlargest


In [174]:

problem_set = [
    [["0", "2"], ["1", "0"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "3"]],
    [["0", "0"], ["1", "1"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "1"], ["1", "3"], ["3", "2"], ["3", "3"]],
    [["0", "2"], ["1", "0"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "1"], ["3", "3"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "3"], ["1", "2"], ["2", "2"], ["2", "3"], ["3", "0"]],
    [["0", "3"], ["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "3"], ["2", "1"], ["2", "3"], ["3", "0"]],
    [["0", "1"], ["0", "3"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "2"]],
    [["0", "0"], ["1", "3"], ["2", "0"], ["2", "1"], ["2", "3"], ["3", "1"]],
    [["0", "1"], ["0", "2"], ["1", "0"], ["1", "2"], ["2", "2"], ["2", "3"]],
    [["0", "3"], ["1", "0"], ["1", "3"], ["2", "1"], ["2", "2"], ["3", "0"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "3"], ["3", "0"], ["3", "1"]],
    [["0", "1"], ["1", "0"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "1"]],
]




def mark_address(qc, qubits, index):

    if index not in range(16):
        raise ValueError("index must be in range(16)")
    
    #if len(qubits) != 4:
    #    raise ValueError("qubits must have length of 4")
    
    # Convert index to a binary string
    bin_str = f'{index:02b}'

    for i in range(len(qubits)):
        if bin_str[i] == "0":
            qc.x(qubits[i])

    return qc


def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    circ = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n // 2):
        circ.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            circ.cp(-np.pi / float(2 ** (j - m)), m, j)
        circ.h(j)

    qft_dag = circ.to_gate()
    qft_dag.name = "$QFT_dag$"
    return qft_dag


def get_game_logic_gate(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_extra, ps):
    
    qc = QuantumCircuit(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_extra)

    for p, puzzle in enumerate(ps): # for each puzzle

        puzzle = np.array(puzzle, int) # convert to int
        # Mark the puzzle's address (p):
        qc = mark_address(qc, qr_addres, p)

        for c, (h, v) in enumerate(puzzle): # for each position pair in puzzle
            # Control cluster c with address and the puzzle's horizontal number
            qc.mct(qr_addres[:] + [qr_shot_h[h]], qr_clustr[c], qr_extra, mode="v-chain")
            qc.mct(qr_addres[:] + [qr_shot_v[v]], qr_clustr[c], qr_extra, mode="v-chain")

        # Unmark the puzzles's address:
        qc = mark_address(qc, qr_addres, p)
    # Convert to gate and return
    game_logic = qc.to_gate()
    game_logic.name = "GameLogic"
    return game_logic


def oracle(qc, qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, ps):

    # Step 1: Enter logic for the game
    game_logic_gate = get_game_logic_gate(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_extra, ps)
    game_qubits = qr_shot_h[:] + qr_shot_v[:] + qr_addres[:] + qr_clustr[:] + qr_extra[:]

    qc.append(game_logic_gate, game_qubits)

    qc.barrier()

    # Step 2: Append counting logic
    qc.h(qr_countr)

    for q_shot in (qr_shot_h[:] + qr_shot_v[:]):
        for i, q_countr in enumerate(qr_countr):
            qc.cp(np.pi / 2 ** (len(qr_countr) - i - 1), q_shot, q_countr)

    qc.append(qft_dagger(len(qr_countr)), qr_countr)

    # Mark the desired state: 3 or less = 00**
    qc.x(qr_countr[2:])

    qc.barrier()

    # Step 3: Check for the solution
    qc.mct(qr_clustr[:] + qr_countr[2:], qr_anclla, qr_extra, mode="v-chain")
    qc.barrier()
    
    # Step 4: Uncompute counting circuit
    # Unmark the desired state
    qc.x(qr_countr[2:])

    # Uncompute counting logic
    qc.append(qft_dagger(len(qr_countr)).inverse(), qr_countr)

    for q_shot in (qr_shot_h[:] + qr_shot_v[:]):
        for i, q_countr in enumerate(qr_countr):
            qc.cp(np.pi / 2 ** (len(qr_countr) - i - 1), q_shot, q_countr)

    qc.h(qr_countr)

    qc.barrier()

    # Step 5: Uncompute game logic
    qc.append(game_logic_gate.inverse(), game_qubits)
        
    qc.barrier()
    
    return qc

def diffusion(qc, qubits, qr_extr):
    qc.h(qubits)
    qc.x(qubits)
    qc.h(qubits[-1])
    qc.mct(qubits[:-1], qubits[-1], qr_extr, mode="recursion")
    qc.h(qubits[-1])
    qc.x(qubits)
    qc.h(qubits)
    
    qc.barrier()

    return qc


def week3_ans_func(problem_set, count_shots=False):
    # Build your quantum circuit here
    # In addition, please make it a function that can solve the 
    # problem even with different inputs (problem_set). We do validation with different inputs.

    # 4 + 4 (shot options) + 6 (clusters) + 4 (address) + 3 (counting) + 1 (ancilla) = 22, 6 extra! 
    qr_shot_h = QuantumRegister(4, name="horz shots")
    qr_shot_v = QuantumRegister(4, name="vert shots")
    qr_clustr = QuantumRegister(6, name="clusters")
    qr_addres = QuantumRegister(2, name="address")
    qr_countr = QuantumRegister(3, name="counting")
    qr_anclla = QuantumRegister(1, name="ancl")
    qr_extra = QuantumRegister(6, name="extra")

    
    cr_shots = ClassicalRegister(8)
    cr_count = ClassicalRegister(3)
    cr_address = ClassicalRegister(2)

    if count_shots:
        qc = QuantumCircuit(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, cr_shots)
    else:
        qc = QuantumCircuit(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, cr_address)
    # Set cluster status to 0
    # No code required

    # Prepare ancilla
    qc.x(qr_anclla)
    qc.h(qr_anclla)

    # Put solution into superposition
    qc.h(qr_shot_h[:] + qr_shot_v[:] + qr_addres[:])

    qc.barrier()
    
    # Code for Grover's algorithm with iterations = 1 will be as follows.
    for i in range(1):
        oracle(qc, qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, problem_set)
        diffusion(qc, qr_shot_h[:] + qr_shot_v[:] + qr_addres[:], qr_extra) # Maybe just address?

    if count_shots:
        qc.measure(qr_shot_h[:] + qr_shot_v[:], cr_shots)
    else:
        qc.measure(qr_addres, cr_address)

    qc = qc.reverse_bits()

    return qc

qc = week3_ans_func(problem_set[8:12], count_shots=True)
# qc.draw(output="mpl")
# New idea: 6 bits for state, 8 bits for shots, problem logic is programmed into oracle? Main question: how to scale?

import time

backend = Aer.get_backend("qasm_simulator")
tic = time.perf_counter()
job = execute(
    qc,
    backend=backend,
    shots=15000,
    backend_options={"fusion_enable": True},
)
toc = time.perf_counter()

print(f"Circuit executed in {toc - tic:0.4f} seconds")

result = job.result()
count = result.get_counts()

for k, v in count.items():
    print(k, v)

Circuit executed in 0.2218 seconds
00000000 44
00000001 57
00010000 53
00010001 69
00010010 74
00010011 74
00010100 64
00010101 50
00010110 87
00010111 47
00011000 62
00011001 57
00011010 58
00011011 46
00011100 79
00011101 55
00011110 49
00011111 58
00000010 59
00100000 48
00100001 77
00100010 55
00100011 60
00100100 71
00100101 73
00100110 63
00100111 61
00101000 71
00101001 63
00101010 68
00101011 51
00101100 59
00101101 54
00101110 49
00101111 64
00000011 69
00110000 76
00110001 55
00110010 63
00110011 46
00110100 64
00110101 45
00110110 44
00110111 50
00111000 40
00111001 45
00111010 41
00111011 60
00111100 56
00111101 62
00111110 69
00111111 73
00000100 55
01000000 62
01000001 81
01000010 77
01000011 57
01000100 62
01000101 55
01000110 53
01000111 43
01001000 62
01001001 70
01001010 50
01001011 39
01001100 54
01001101 51
01001110 61
01001111 61
00000101 52
01010000 65
01010001 56
01010010 70
01010011 57
01010100 70
01010101 57
01010110 54
01010111 48
01011000 65
01011001 60
01011

Circuit executed in 0.2100 seconds
00 3876
01 3670
10 3795
11 3659


In [167]:

problem_set = [
    [["0", "2"], ["1", "0"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "3"]],
    [["0", "0"], ["1", "1"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "1"], ["1", "3"], ["3", "2"], ["3", "3"]],
    [["0", "2"], ["1", "0"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "1"], ["3", "3"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "3"], ["1", "2"], ["2", "2"], ["2", "3"], ["3", "0"]],
    [["0", "3"], ["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "3"], ["2", "1"], ["2", "3"], ["3", "0"]],
    [["0", "1"], ["0", "3"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "2"]],
    [["0", "0"], ["1", "3"], ["2", "0"], ["2", "1"], ["2", "3"], ["3", "1"]],
    [["0", "1"], ["0", "2"], ["1", "0"], ["1", "2"], ["2", "2"], ["2", "3"]],
    [["0", "3"], ["1", "0"], ["1", "3"], ["2", "1"], ["2", "2"], ["3", "0"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "3"], ["3", "0"], ["3", "1"]],
    [["0", "1"], ["1", "0"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "1"]],
]




def mark_address(qc, qubits, index):

    if index not in range(16):
        raise ValueError("index must be in range(16)")
    
    #if len(qubits) != 4:
    #    raise ValueError("qubits must have length of 4")
    
    # Convert index to a binary string
    bin_str = f'{index:02b}'

    for i in range(len(qubits)):
        if bin_str[i] == "0":
            qc.x(qubits[i])

    return qc


def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    circ = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n // 2):
        circ.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            circ.cp(-np.pi / float(2 ** (j - m)), m, j)
        circ.h(j)

    qft_dag = circ.to_gate()
    qft_dag.name = "$QFT_dag$"
    return qft_dag


def get_game_logic_gate(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_extra, ps):
    
    qc = QuantumCircuit(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_extra)

    for p, puzzle in enumerate(ps): # for each puzzle

        puzzle = np.array(puzzle, int) # convert to int
        # Mark the puzzle's address (p):
        qc = mark_address(qc, qr_addres, p)

        for c, (h, v) in enumerate(puzzle): # for each position pair in puzzle
            # Control cluster c with address and the puzzle's horizontal number
            qc.mct(qr_addres[:] + [qr_shot_h[h]], qr_clustr[c], qr_extra, mode="v-chain")
            qc.mct(qr_addres[:] + [qr_shot_v[v]], qr_clustr[c], qr_extra, mode="v-chain")

        # Unmark the puzzles's address:
        qc = mark_address(qc, qr_addres, p)
    # Convert to gate and return
    game_logic = qc.to_gate()
    game_logic.name = "GameLogic"
    return game_logic


def game_oracle(qc, qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, ps):

    # Step 1: Enter logic for the game
    game_logic_gate = get_game_logic_gate(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_extra, ps)
    game_qubits = qr_shot_h[:] + qr_shot_v[:] + qr_addres[:] + qr_clustr[:] + qr_extra[:]

    qc.append(game_logic_gate, game_qubits)

    qc.barrier()

    # Step 2: Check for the solution
    qc.mct(qr_clustr[:], qr_anclla, qr_extra, mode="v-chain")
    qc.barrier()
    
    # Step 3: Uncompute game logic
    qc.append(game_logic_gate.inverse(), game_qubits)
        
    qc.barrier()
    
    return qc

def count_oracle(qc, qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, ps):

    # Step 1: Append counting logic
    qc.h(qr_countr)

    for q_shot in (qr_shot_h[:] + qr_shot_v[:]):
        for i, q_countr in enumerate(qr_countr):
            qc.cp(np.pi / 2 ** (len(qr_countr) - i - 1), q_shot, q_countr)

    qc.append(qft_dagger(len(qr_countr)), qr_countr)

    # Mark the desired state: 3 or less = 00**
    qc.x(qr_countr[2:])

    qc.barrier()

    # Step 2: Check for the solution
    qc.mct(qr_countr, qr_anclla, qr_extra, mode="v-chain")
    qc.barrier()
    
    # Step 3: Uncompute counting circuit
    # Unmark the desired state
    qc.x(qr_countr[2:])

    # Uncompute counting logic
    qc.append(qft_dagger(len(qr_countr)).inverse(), qr_countr)

    for q_shot in (qr_shot_h[:] + qr_shot_v[:]):
        for i, q_countr in enumerate(qr_countr):
            qc.cp(np.pi / 2 ** (len(qr_countr) - i - 1), q_shot, q_countr)

    qc.h(qr_countr)

    qc.barrier()
    
    return qc

def diffusion(qc, qubits, qr_extr):
    qc.h(qubits)
    qc.x(qubits)
    qc.h(qubits[-1])
    qc.mct(qubits[:-1], qubits[-1], qr_extr, mode="recursion")
    qc.h(qubits[-1])
    qc.x(qubits)
    qc.h(qubits)
    
    qc.barrier()

    return qc


def week3_ans_func_2(problem_set, count_shots=False):
    # Build your quantum circuit here
    # In addition, please make it a function that can solve the 
    # problem even with different inputs (problem_set). We do validation with different inputs.   

    # 4 + 4 (shot options) + 6 (clusters) + 4 (address) + 3 (counting) + 1 (ancilla) = 22, 6 extra! 
    qr_shot_h = QuantumRegister(4, name="horz shots")
    qr_shot_v = QuantumRegister(4, name="vert shots")
    qr_clustr = QuantumRegister(6, name="clusters")
    qr_addres = QuantumRegister(2, name="address")
    qr_countr = QuantumRegister(3, name="counting")
    qr_anclla = QuantumRegister(1, name="ancl")
    qr_extra = QuantumRegister(6, name="extra")
    
    cr_shots = ClassicalRegister(8)
    cr_count = ClassicalRegister(3)
    cr_address = ClassicalRegister(2)

    if count_shots:
        qc = QuantumCircuit(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, cr_shots)
    else:
        qc = QuantumCircuit(qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, cr_address)
    
    # Set cluster status to 0
    # No code required

    # Prepare ancilla
    qc.x(qr_anclla)
    qc.h(qr_anclla)

    # Put solution into superposition
    qc.h(qr_shot_h[:] + qr_shot_v[:] + qr_addres[:])

    qc.barrier()
    
    # Code for Grover's algorithm with iterations = 1 will be as follows.
    for i in range(1):
        qc = game_oracle(qc, qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, problem_set)
        qc = diffusion(qc, qr_shot_h[:] + qr_shot_v[:], qr_extra) # Maybe just address?
        #qc = count_oracle(qc, qr_shot_h, qr_shot_v, qr_addres, qr_clustr, qr_countr, qr_anclla, qr_extra, problem_set)
        #qc = diffusion(qc, qr_shot_h[:] + qr_shot_v[:] + qr_addres[:], qr_extra) # Maybe just address?
        # diffusion(qc, qr_addres, qr_extra)

    if count_shots:
        qc.measure(qr_shot_h[:] + qr_shot_v[:], cr_shots)
    else:
        qc.measure(qr_addres, cr_address)
    
    qc = qc.reverse_bits()

    return qc

qc = week3_ans_func_2(problem_set[8:12], count_shots=True)

import time

backend = Aer.get_backend("qasm_simulator")
tic = time.perf_counter()
job = execute(
    qc,
    backend=backend,
    shots=15000,
    backend_options={"fusion_enable": True},
)
toc = time.perf_counter()

print(f"Circuit executed in {toc - tic:0.4f} seconds")

result = job.result()
count = result.get_counts()

for k, v in count.items():
    print(k, v)


Circuit executed in 1.7248 seconds
00000000 56
00000001 49
00010000 55
00010001 49
00010010 61
00010011 59
00010100 38
00010101 52
00010110 46
00010111 52
00011000 44
00011001 61
00011010 52
00011011 57
00011100 65
00011101 39
00011110 44
00011111 46
00000010 42
00100000 58
00100001 59
00100010 50
00100011 52
00100100 48
00100101 64
00100110 42
00100111 150
00101000 59
00101001 45
00101010 60
00101011 65
00101100 56
00101101 56
00101110 51
00101111 37
00000011 49
00110000 33
00110001 37
00110010 44
00110011 49
00110100 51
00110101 49
00110110 55
00110111 63
00111000 48
00111001 49
00111010 62
00111011 53
00111100 37
00111101 42
00111110 58
00111111 58
00000100 57
01000000 54
01000001 40
01000010 45
01000011 50
01000100 65
01000101 49
01000110 51
01000111 72
01001000 45
01001001 53
01001010 37
01001011 38
01001100 56
01001101 48
01001110 57
01001111 47
00000101 47
01010000 53
01010001 31
01010010 47
01010011 45
01010100 53
01010101 55
01010110 46
01010111 50
01011000 60
01011001 40
0101